# Task-03 基于机器学习的文本分类
在上一章节，我们对赛题的数据进行了读取，并在末尾给出了两个小作业。如果你顺利完成了作业，那么你基本上对Python也比较熟悉了。在本章我们将使用传统机器学习算法来完成新闻分类的过程，将会结束到赛题的核心知识点。

## 基于机器学习的文本分类
在本章我们将开始使用机器学习模型来解决文本分类。机器学习发展比较广，且包括多个分支，本章侧重使用传统机器学习，从下一章开始是基于深度学习的文本分类。

## 学习目标
- 学会TF-IDF的原理和使用
- 使用sklearn的机器学习模型完成文本分类

## 机器学习模型
机器学习是对能通过经验自动改进的计算机算法的研究。机器学习通过历史数据**训练**出**模型**对应于人类对经验进行**归纳**的过程，机器学习利用**模型**对新数据进行**预测**对应于人类利用总结的**规律**对新问题进行**预测**的过程。

机器学习有很多种分支，对于学习者来说应该优先掌握机器学习算法的分类，然后再其中一种机器学习算法进行学习。由于机器学习算法的分支和细节实在是太多，所以如果你一开始就被细节迷住了眼，你就很难知道全局是什么情况的。

如果你是机器学习初学者，你应该知道如下的事情：

1. 机器学习能解决一定的问题，但不能奢求机器学习是万能的；
2. 机器学习算法有很多种，看具体问题需要什么，再来进行选择；
3. 每种机器学习算法有一定的偏好，需要具体问题具体分析；

## 文本表示方法 Part1
在机器学习算法的训练过程中，假设给定$N$个样本，每个样本有$M$个特征，这样组成了$N×M$的样本矩阵，然后完成算法的训练和预测。同样的在计算机视觉中可以将图片的像素看作特征，每张图片看作$hight×width×3$的特征图，一个三维的矩阵来进入计算机进行计算。

但是在自然语言领域，上述方法却不可行：文本是不定长度的。文本表示成计算机能够运算的数字或向量的方法一般称为词嵌入（Word Embedding）方法。词嵌入将不定长的文本转换到定长的空间内，是文本分类的第一步。

### One-hot
这里的One-hot与数据挖掘任务中的操作是一致的，即将每一个单词使用一个离散的向量表示。具体将每个字/词编码一个索引，然后根据索引进行赋值。

One-hot表示方法的例子如下：

句子1：我 爱 北 京 天 安 门

句子2：我 喜 欢 上 海

首先对所有句子的字进行索引，即将每个字确定一个编号：

{
    '我': 1, '爱': 2, '北': 3, '京': 4, '天': 5,
  '安': 6, '门': 7, '喜': 8, '欢': 9, '上': 10, '海': 11
}

在这里共包括11个字，因此每个字可以转换为一个11维度稀疏向量：

我：[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

爱：[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]

...

海：[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]

### Bag of Words
Bag of Words（词袋表示），也称为Count Vectors，每个文档的字/词可以使用其出现次数来进行表示。

句子1：我 爱 北 京 天 安 门

句子2：我 喜 欢 上 海

直接统计每个字出现的次数，并进行赋值：

句子1：我 爱 北 京 天 安 门

转换为 [1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0]

句子2：我 喜 欢 上 海

转换为 [1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1]

在sklearn中可以直接```CountVectorizer```来实现这一步骤：

```python
from sklearn.feature_extraction.text import CountVectorizer
corpus = [
    'This is the first document.',
    'This document is the second document.',
    'And this is the third one.',
    'Is this the first document?',
]
vectorizer = CountVectorizer()
vectorizer.fit_transform(corpus).toarray()
```

In [4]:
from sklearn.feature_extraction.text import CountVectorizer
corpus = [
    'This is the first document.',
    'This document is the second document.',
    'And this is the third one.',
    'Is this the first document?',
]
vectorizer = CountVectorizer()
vectorizer.fit_transform(corpus).toarray()

array([[0, 1, 1, 1, 0, 0, 1, 0, 1],
       [0, 2, 0, 1, 0, 1, 1, 0, 1],
       [1, 0, 0, 1, 1, 0, 1, 1, 1],
       [0, 1, 1, 1, 0, 0, 1, 0, 1]], dtype=int64)

### N-gram
N-gram与Count Vectors类似，不过加入了相邻单词组合成为新的单词，并进行计数。

如果N取值为2，则句子1和句子2就变为：

句子1：我爱 爱北 北京 京天 天安 安门

句子2：我喜 喜欢 欢上 上海

### TF-IDF
TF-IDF 分数由两部分组成：第一部分是词语频率（Term Frequency），第二部分是逆文档频率（Inverse Document Frequency）。其中计算语料库中文档总数除以含有该词语的文档数量，然后再取对数就是逆文档频率。

    TF(t)= 该词语在当前文档出现的次数 / 当前文档中词语的总数
    IDF(t)= log_e（文档总数 / 出现该词语的文档总数）

## 基于机器学习的文本分类
接下来我们将对比不同文本表示算法的精度，通过本地构建验证集计算F1得分。

In [7]:
# Count Vectors + RidgeClassifier

import pandas as pd

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import RidgeClassifier
from sklearn.metrics import f1_score

train_df = pd.read_csv('./input/train_set.csv', sep='\t', nrows=15000)

vectorizer = CountVectorizer(max_features=3000)
train_test = vectorizer.fit_transform(train_df['text'])

clf = RidgeClassifier()
clf.fit(train_test[:10000], train_df['label'].values[:10000])

val_pred = clf.predict(train_test[10000:])
print(f1_score(train_df['label'].values[10000:], val_pred, average='macro'))
# 0.74

0.7406241569237678


In [9]:
# TF-IDF +  RidgeClassifier

import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import RidgeClassifier
from sklearn.metrics import f1_score

train_df = pd.read_csv('./input/train_set.csv', sep='\t', nrows=15000)

tfidf = TfidfVectorizer(ngram_range=(1,3), max_features=3000)
train_test = tfidf.fit_transform(train_df['text'])

clf = RidgeClassifier()
clf.fit(train_test[:10000], train_df['label'].values[:10000])

val_pred = clf.predict(train_test[10000:])
print(f1_score(train_df['label'].values[10000:], val_pred, average='macro'))
# 0.87

0.8721598830546126


## 本章小结
本章我们介绍了基于机器学习的文本分类方法，并完成了两种方法的对比。

## 本章作业
1. 尝试改变TF-IDF的参数，并验证精度
2. 尝试使用其他机器学习模型，完成训练和验证

In [37]:
# TF-IDF +  RidgeClassifier

import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import RidgeClassifier
from sklearn.metrics import f1_score

train_df = pd.read_csv('./input/train_set.csv', sep='\t', nrows=15000)

tfidf = TfidfVectorizer(stop_words=['3750'], ngram_range=(1,1), max_features=3000, max_df=0.8)
train_test = tfidf.fit_transform(train_df['text'])

clf = RidgeClassifier()
clf.fit(train_test[:10000], train_df['label'].values[:10000])

val_pred = clf.predict(train_test[10000:])
print(f1_score(train_df['label'].values[10000:], val_pred, average='macro'))
# 0.87

0.856442556300544


vectorizer = CountVectorizer() #构建一个计算词频（TF）的玩意儿，当然这里面不足是可以做这些

transformer = TfidfTransformer() #构建一个计算TF-IDF的玩意儿

tfidf = transformer.fit_transform(vectorizer.fit_transform(corpus))

#vectorizer.fit_transform(corpus)将文本corpus输入，得到词频矩阵

#将这个矩阵作为输入，用transformer.fit_transform(词频矩阵)得到TF-IDF权重矩阵



    TfidfTransformer + CountVectorizer  =  TfidfVectorizer



值得注意的是，CountVectorizer()和TfidfVectorizer()里面都有一个成员叫做vocabulary_（后面带一个下划线）

这个成员的意义是词典索引，对应的是TF-IDF权重矩阵的列，只不过一个是私有成员，一个是外部输入，原则上应该保持一致。



    vectorizer = TfidfVectorizer(stop_words=stpwrdlst, sublinear_tf = True, max_df = 0.5)



关于参数：

**input** ：string{'filename', 'file', 'content'}

    如果是'filename'，序列作为参数传递给拟合器，预计为文件名列表，这需要读取原始内容进行分析

    如果是'file'，序列项目必须有一个”read“的方法（类似文件的对象），被调用作为获取内存中的字节数

    否则，输入预计为序列串，或字节数据项都预计可直接进行分析。

**encoding** ：string， ‘utf-8’by default

    如果给出要解析的字节或文件，此编码将用于解码

**decode_error** : {'strict', 'ignore', 'replace'}

    如果一个给出的字节序列包含的字符不是给定的编码，指示应该如何去做。默认情况下，它是'strict'，这意味着的UnicodeDecodeError将提高，其他值是'ignore'和'replace'

**strip_accents** : {'ascii', 'unicode', None}

    在预处理步骤中去除编码规则(accents)，”ASCII码“是一种快速的方法，仅适用于有一个直接的ASCII字符映射，"unicode"是一个稍慢一些的方法，None（默认）什么都不做

**analyzer** ：string，{'word', 'char'} or callable

    定义特征为词（word）或n-gram字符，如果传递给它的调用被用于抽取未处理输入源文件的特征序列

**preprocessor** ：callable or None（default）

    当保留令牌和”n-gram“生成步骤时，覆盖预处理（字符串变换）的阶段

**tokenizer** ：callable or None(default)

    当保留预处理和n-gram生成步骤时，覆盖字符串令牌步骤

**ngram_range** : tuple(min_n, max_n)

    要提取的n-gram的n-values的下限和上限范围，在min_n <= n <= max_n区间的n的全部值
    
    ngram_range(min,max)是指将text分成min，min+1，min+2,.........max 个不同的词组

比如'Python is useful'中ngram_range(1,3)之后可得到'Python'  'is'  'useful'  'Python is'  'is useful' 和'Python is useful'如果是ngram_range (1,1) 则只能得到单个单词'Python'  'is'和'useful'


**stop_words** ：string {'english'}, list, or None(default)

    如果未english，用于英语内建的停用词列表

    如果未list，该列表被假定为包含停用词，列表中的所有词都将从令牌中删除

    如果None，不使用停用词。max_df可以被设置为范围[0.7, 1.0)的值，基于内部预料词频来自动检测和过滤停用词

**lowercase** ：boolean， default True

    在令牌标记前转换所有的字符为小写

**token_pattern** ：string

    正则表达式显示了”token“的构成，仅当analyzer == ‘word’时才被使用。两个或多个字母数字字符的正则表达式（标点符号完全被忽略，始终被视为一个标记分隔符）。

**max_df** ： float in range [0.0, 1.0] or int, optional, 1.0 by default

    当构建词汇表时，严格忽略高于给出阈值的文档频率的词条，语料指定的停用词。如果是浮点值，该参数代表文档的比例，整型绝对计数值，如果词汇表不为None，此参数被忽略。

**min_df** ：float in range [0.0, 1.0] or int, optional, 1.0 by default

当构建词汇表时，严格忽略低于给出阈值的文档频率的词条，语料指定的停用词。如果是浮点值，该参数代表文档的比例，整型绝对计数值，如果词汇表不为None，此参数被忽略。

**max_features** ： optional， None by default

    如果不为None，构建一个词汇表，仅考虑max_features--按语料词频排序，如果词汇表不为None，这个参数被忽略

**vocabulary** ：Mapping or iterable， optional

    也是一个映射（Map）（例如，字典），其中键是词条而值是在特征矩阵中索引，或词条中的迭代器。如果没有给出，词汇表被确定来自输入文件。在映射中索引不能有重复，并且不能在0到最大索引值之间有间断。

**binary** ：boolean， False by default

    如果未True，所有非零计数被设置为1，这对于离散概率模型是有用的，建立二元事件模型，而不是整型计数

**dtype** ：type， optional

    通过fit_transform()或transform()返回矩阵的类型

**norm** ：'l1', 'l2', or None,optional

    范数用于标准化词条向量。None为不归一化

**use_idf** ：boolean， optional

    启动inverse-document-frequency重新计算权重

**smooth_idf** ：boolean，optional

    通过加1到文档频率平滑idf权重，为防止除零，加入一个额外的文档

**sublinear_tf** ：boolean， optional

    应用线性缩放TF，例如，使用1+log(tf)覆盖tf


In [33]:
# Count Vectors + RidgeClassifier

import pandas as pd

from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import RidgeClassifier
from sklearn.metrics import f1_score

train_df = pd.read_csv('./input/train_set.csv', sep='\t', nrows=15000)

enc = OneHotEncoder()
train_test = enc.fit(train_df[['text']])

# vectorizer = CountVectorizer(max_features=3000)
# train_test = vectorizer.fit_transform(train_df['text'])

clf = RidgeClassifier()
clf.fit(train_test, train_df[['label']].values)

val_pred = clf.predict(train_test[10000:])
print(f1_score(train_df['label'].values[10000:], val_pred, average='macro'))

AttributeError: 'OneHotEncoder' object has no attribute 'ndim'

In [38]:
# HashingVectorizer +  RidgeClassifier

import pandas as pd

from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.linear_model import RidgeClassifier
from sklearn.metrics import f1_score

train_df = pd.read_csv('./input/train_set.csv', sep='\t', nrows=15000)

tfidf = HashingVectorizer(stop_words=['3750'], ngram_range=(1,3))
train_test = tfidf.fit_transform(train_df['text'])

clf = RidgeClassifier()
clf.fit(train_test[:10000], train_df['label'].values[:10000])

val_pred = clf.predict(train_test[10000:])
print(f1_score(train_df['label'].values[10000:], val_pred, average='macro'))
# 0.87

0.8902579704497026


In [39]:
# TF-IDF +  RidgeClassifierCV

import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import RidgeClassifierCV
from sklearn.metrics import f1_score

train_df = pd.read_csv('./input/train_set.csv', sep='\t', nrows=15000)

tfidf = TfidfVectorizer(ngram_range=(1,3), max_features=3000)
train_test = tfidf.fit_transform(train_df['text'])

clf = RidgeClassifierCV()
clf.fit(train_test[:10000], train_df['label'].values[:10000])

val_pred = clf.predict(train_test[10000:])
print(f1_score(train_df['label'].values[10000:], val_pred, average='macro'))
# 0.87

0.8719372173702


In [42]:
# TF-IDF +  LogisticRegression

import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score

train_df = pd.read_csv('./input/train_set.csv', sep='\t', nrows=15000)

tfidf = TfidfVectorizer(ngram_range=(1,3), max_features=3000)
train_test = tfidf.fit_transform(train_df['text'])

clf = LogisticRegression(max_iter=3000)
clf.fit(train_test[:10000], train_df['label'].values[:10000])

val_pred = clf.predict(train_test[10000:])
print(f1_score(train_df['label'].values[10000:], val_pred, average='macro'))
# 0.87

0.8498842103097133
